# Kermack-McKendrick Model 

#### Code:
- http://greenteapress.com/modsimpy/ModSimPy3.pdf
- https://github.com/AllenDowney/ModSimPy/blob/master/notebooks/chap11.ipynb

#### Model:
- https://mathworld.wolfram.com/Kermack-McKendrickModel.html

In [1]:
import pandas as pd
from modsim import *

# Functions

In [20]:
def make_system(beta, gamma,duration=2*365,S=7000000000, I=500000, R=123000):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(S=S, I=I, R=R)
    init /= sum(init)

    t0 = 0
    t_end = duration

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma)

In [44]:
def update_func(state, t, system):
    """Update the SIR model.
    
    state: State with variables S, I, R
    t: time step
    system: System with beta and gamma
    
    returns: State object
    """
    s, i, r = state

    infected = system.beta * i * s    
    recovered = system.gamma * i
    
    s -= infected * t
    i += (infected - recovered) * t
    r += recovered * t
    
    return State(S=s, I=i, R=r)

In [45]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    frame = TimeFrame(columns=system.init.index)
    frame.row[system.t0] = system.init
    
    for t in linrange(system.t0, system.t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame

# Run

- vary tc and tr

In [70]:
tc = 7      # time between contacts in days 
tr = 45      # recovery time in days

beta = 1 / tc      # contact rate in per day
gamma = 1 / tr     # recovery rate in per day

duration=60
S=7000000000 # population
I=500000 # infected as of march 26, 2020
R=123000 # recovered (excl. deaths) as of march 26, 2020


system = make_system(beta, gamma,duration,S,I,R)
results = run_simulation(system, update_func)
results

,S,I,R
0,0.999911,7.142221e-05,0.000018
1,0.999911,7.142221e-05,0.000018
2,0.999901,8.003732e-05,0.000019
3,0.999878,9.934564e-05,0.000023
4,0.999835,1.352941e-04,0.000029
...,...,...,...
56,0.002325,7.233978e-30,0.997675
57,0.002325,-1.633781e-30,0.997675
58,0.002325,4.047503e-31,0.997675
59,0.002325,-1.091322e-31,0.997675


# Plot results

In [71]:
results.reset_index(inplace=True)

In [72]:
death_rate=0.1
results['Deaths']=death_rate*results['I']

In [73]:
#results['R_0']=(beta/gamma)*results['I']

In [74]:
df = results.melt(id_vars='index')

In [75]:
import plotly.express as px

fig = px.line(df, x="index", y="value", color="variable", line_group="variable", hover_name="variable",
        line_shape="spline", render_mode="svg")
fig.show()